In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import os

from main import DCTrader, Backtest, DCTracker, StockDataLoader
from trainer import GATrainer, PSOTrainer

In [3]:
ticker = 'test'

test = StockDataLoader(ticker=ticker)
test.load_data()

test: 49 loaded days | 2020-01-01 to 2020-02-18


In [4]:
test.data

,Date,Close
0,2020-01-01,99.9
1,2020-01-02,98.0
2,2020-01-03,97.0
3,2020-01-04,94.0
4,2020-01-05,98.0
5,2020-01-06,102.0
6,2020-01-07,104.0
7,2020-01-08,103.0
8,2020-01-09,109.0
9,2020-01-10,110.0


In [5]:
tracker = DCTracker(theta=0.06)

dc_confirmations = []

for idx, row in test.data.iterrows():
    t = idx  # Tiempo como índice entero
    p = row['Close']
    tracker.update(t, p)
    
    if tracker.is_dcc:
        dc_confirmations.append({
            'index': idx,
            'date': row['Date'].date(),
            'price': p,
            'new_trend': tracker.trend,
            'from_ext': tracker.p_ext_initial
        })

print("\n=== DC CONFIRMATIONS DETECTADAS (theta=6%) ===")
for c in dc_confirmations:
    print(f"Index {c['index']:2d} | {c['date']} | Precio {c['price']:6.2f} → {c['new_trend'].upper()} "
          f"(desde extremum {c['from_ext']:.2f})")


=== DC CONFIRMATIONS DETECTADAS (theta=6%) ===
Index  5 | 2020-01-06 | Precio 102.00 → UPTREND (desde extremum 94.00)
Index 12 | 2020-01-13 | Precio  96.00 → DOWNTREND (desde extremum 110.00)
Index 19 | 2020-01-20 | Precio 100.00 → UPTREND (desde extremum 90.00)
Index 26 | 2020-01-27 | Precio 107.00 → DOWNTREND (desde extremum 115.00)
Index 36 | 2020-02-06 | Precio  95.00 → UPTREND (desde extremum 85.00)
Index 43 | 2020-02-13 | Precio  94.00 → DOWNTREND (desde extremum 102.00)
Index 44 | 2020-02-14 | Precio 105.00 → UPTREND (desde extremum 94.00)
Index 47 | 2020-02-17 | Precio  99.00 → DOWNTREND (desde extremum 110.00)


In [6]:
print("=== ESTADO FINAL DEL TRACKER ===")
print(tracker)

=== ESTADO FINAL DEL TRACKER ===
=== DCTracker State ===
Theta: 0.0600
Current trend: downtrend
Current extreme price (p_ext): 97.000000
Time of extreme (t_ext): 48
Theoretical DCC (p_dcc_star): 103.400000
Initial extreme cycle: 110.000000
Last DCC time: 47
Current OS length: 1
Last DC duration: 1
Is DCC now?: False

=== Statistics ===
DC events: 8
OS events: 7
Total DC time: 17
Total OS time: 30

OSV DT (4): [0.6129, 0.548, 2.0197, 1.5853]
OSV UT (4): [0.6089, 2.0266, 0.7214, 0.1071]
TMV DT (4): [0.3504, 1.5152, 2.8986, 0.4902]
TMV UT (4): [0.3546, 3.1481, 1.7647, 0.8865]

Recent trends: ['uptrend', 'downtrend', 'uptrend', 'downtrend', 'uptrend']
Recent OS flags: [True, True, True, False, True]
